In [5]:
import pandas as pd
import MeCab, re, json, glob
tagger = MeCab.Tagger()

# replace \n+ with ''
# 0:surface form, 1:pos, 2:pos2(サ変接続, etc), 3:pos3, 4:pos4
# 5:conjugation type, 6:conjugation form, 7:lemma, 8:phono, 9:phone

def tokenize_mecab(text:str) -> list:
    text = re.sub(r'\n+', '\n', text)
    tokens = tagger.parse(text).split('\n')[:-2]
    return [re.split(r'[,\t]', token) for token in tokens]

def get_verb_group(conjtype:str) -> str:
    """
    >>> get_verb_group('五段・カ行')
    '1'
    """
    return {'五段':'1', '一段':'2', 'サ変':'3', 'カ変':'3'}.get(conjtype[:2], '')

def tokenize(text:str) -> list:
    text = re.sub(r'\n+', '\n', text)
    tokens = tagger.parse(text).split('\n')[:-2]
    tokens = [re.split(r'[,\t]', token) for token in tokens]
    surface, lemma, pos = ['<SEP>'],['<SEP>'],['<SEP>']
    
    i = 0
    while i < len(tokens):
        if tokens[i][2] == 'サ変接続' and tokens[i+1][2] == 'サ変接続' and tokens[i+2][7] == 'する':
            surface.append(tokens[i][0] + tokens[i+1][0] + tokens[i+2][0]) # surface form
            lemma.append(tokens[i][7] + tokens[i+1][7] + tokens[i+2][0]) # lemma
            pos.append('動詞3') # pos
            i += 2
        elif tokens[i][2] == 'サ変接続' and tokens[i+1][7] == 'する':
            surface.append(tokens[i][0] + tokens[i+1][0]) # surface form
            lemma.append(tokens[i][7] + tokens[i+1][7]) # lemma
            pos.append('動詞3') # pos
            i += 2
        else:
            surface.append(tokens[i][0]) # surface form
            lemma.append(tokens[i][7]) # lemma
            pos.append(tokens[i][1]) # pos  
            if tokens[i][0] == '。':
                surface.append('<SEP>')
                lemma.append('<SEP>')
                pos.append('<SEP>')
            i += 1
    return surface, lemma, pos

In [2]:
tokenize_mecab('運動させる際は予防対策を選挙活動するよう求めています。')

[['運動', 'ウンドー', 'ウンドウ', '運動', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['さ', 'サ', 'スル', '為る', '動詞-非自立可能', 'サ行変格', '未然形-サ', '0'],
 ['せる', 'セル', 'セル', 'せる', '助動詞', '下一段-サ行', '連体形-一般', ''],
 ['際', 'サイ', 'サイ', '際', '名詞-普通名詞-副詞可能', '', '', '1'],
 ['は', 'ワ', 'ハ', 'は', '助詞-係助詞', '', '', ''],
 ['予防', 'ヨボー', 'ヨボウ', '予防', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['対策', 'タイサク', 'タイサク', '対策', '名詞-普通名詞-一般', '', '', '0'],
 ['を', 'オ', 'ヲ', 'を', '助詞-格助詞', '', '', ''],
 ['選挙', 'センキョ', 'センキョ', '選挙', '名詞-普通名詞-サ変可能', '', '', '1'],
 ['活動', 'カツドー', 'カツドウ', '活動', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['する', 'スル', 'スル', '為る', '動詞-非自立可能', 'サ行変格', '連体形-一般', '0'],
 ['よう', 'ヨー', 'ヨウ', '様', '形状詞-助動詞語幹', '', '', '1'],
 ['求め', 'モトメ', 'モトメル', '求める', '動詞-一般', '下一段-マ行', '連用形-一般', '3'],
 ['て', 'テ', 'テ', 'て', '助詞-接続助詞', '', '', ''],
 ['い', 'イ', 'イル', '居る', '動詞-非自立可能', '上一段-ア行', '連用形-一般', '0'],
 ['ます', 'マス', 'マス', 'ます', '助動詞', '助動詞-マス', '終止形-一般', ''],
 ['。', '', '', '。', '補助記号-句点', '', '', '']]

In [68]:
tokenize('運動させる際は予防対策を選挙活動するよう求めています。')

(['<SEP>',
  '運動さ',
  'せる',
  '際',
  'は',
  '予防',
  '対策',
  'を',
  '選挙活動する',
  'する',
  'よう',
  '求め',
  'て',
  'い',
  'ます',
  '。',
  '<SEP>'],
 ['<SEP>',
  '運動する',
  'せる',
  '際',
  'は',
  '予防',
  '対策',
  'を',
  '選挙活動する',
  'する',
  'よう',
  '求める',
  'て',
  'いる',
  'ます',
  '。',
  '<SEP>'],
 ['<SEP>',
  '動詞3',
  '動詞',
  '名詞',
  '助詞',
  '名詞',
  '名詞',
  '助詞',
  '動詞3',
  '動詞',
  '名詞',
  '動詞',
  '助詞',
  '動詞',
  '助動詞',
  '記号',
  '<SEP>'])

In [7]:
jsons = sorted(glob.glob('nhkweb20*.json'))
jsons

['nhkweb2012.json',
 'nhkweb2013.json',
 'nhkweb2014.json',
 'nhkweb2015.json',
 'nhkweb2016.json',
 'nhkweb2017.json',
 'nhkweb2018.json',
 'nhkweb2019.json',
 'nhkweb2020.json']

In [9]:
# all NHK data

df = pd.read_json(jsons[0])
for i in range(1, len(jsons)):
    df = pd.concat([df, pd.read_json(jsons[i])])
df = df[['id', 'title', 'article', 'genre', 'datePublished']]
df['article'] = df.article.apply(lambda x:re.sub(r'\n+', '', x))
#df['datePublished'] = df.datePublished.apply(lambda x:x.split('T')[0])
df

,id,title,article,genre,datePublished
0,k10013902141000,“原発政策ころころ変わる”,野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策につ...,[政治],2012-12-02T18:09
1,k10014002951000,六本木襲撃事件 １５人に逮捕状,ことし９月、東京・六本木のクラブで、客の男性が男らに金属バットのようなもので殴られて死亡した...,[社会],2012-12-06T16:13
2,k10014085401000,専門家“ミサイル内部の機器不具合か”,北朝鮮が発射を予告した事実上の弾道ミサイルを、１０日、発射しなかったことについて、海上自衛隊...,[社会],2012-12-10T15:51
3,k10014344461000,新華社“島巡る争いが試練”,中国国営の新華社通信は、ことしの中国外交について、日本政府による沖縄県の尖閣諸島の国有化や南...,[国際],2012-12-21T12:01
4,k10014441691000,鉄板直撃２人死亡 会社を捜索,２５日、広島県東広島市の国道を走っていた大型トレーラーの荷台から崩れ落ちた鉄板が乗用車を直撃...,[社会],2012-12-26T13:49
...,...,...,...,...,...
24538,k10012505311000,沖縄 石垣 新たに1人感染確認 新型コロナウイルス,沖縄県は、新たに石垣市に住む50代の女性が新型コロナウイルスに感染していることが確認されたと...,[社会],2020-07-08T21:53:13+09:00
24539,k10012505321000,長崎大学病院に入院の60代男性 感染確認 新型コロナウイルス,長崎市と長崎大学病院は、大学病院の消化器内科に入院していた60代の男性が、新型コロナウイルス...,[社会],2020-07-08T22:20:57+09:00
24540,k10012505351000,サッカーＪ1 第3節9試合の結果 セレッソ大阪が唯一の3連勝,サッカーＪ1は第3節の9試合が行われ、セレッソ大阪が清水エスパルスに勝ち、開幕から中断期間を...,[スポーツ],2020-07-08T23:10:39+09:00
24541,k10012505361000,長野 松本 上高地で219人が孤立状態,松本市によりますと上高地では、国道158号が土砂崩れで通れなくなり、8日午後5時の時点で5つ...,[気象・災害],2020-07-08T23:16:30+09:00


In [11]:
easy = pd.read_json('nhkwebeasy.json')
easy

,id,title_easy,title_easy_ruby,article_easy,article_easy_ruby,url_easy,url_normal,date_easy
0,k10010000621000,ウィリアム王子 地震や津波で亡くなった人に花を贈る,ウィリアム<ruby>王子<rt>おうじ</rt></ruby> <ruby>地震<rt>じ...,<plc>イギリス</plc>の<per>エリザベス</per>女王の孫、<per>ウィリア...,"<span class=""colorL"">イギリス</span>の<span class=""...",http://www3.nhk.or.jp/news/easy/k1001000062100...,http://www3.nhk.or.jp/news/html/20150301/k1001...,03月02日 17時30分
1,k10010000861000,「流しびな」で子どもが元気に大きくなるように願う,「<ruby>流<rt>なが</rt></ruby>しびな」で<ruby>子<rt>こ</r...,３月３日はひな祭りです。１日、<plc>埼玉県</plc><plc>さいたま市</plc>の...,３<ruby>月<rt>がつ</rt></ruby><ruby>３日<rt>みっか</rt>...,http://www3.nhk.or.jp/news/easy/k1001000086100...,http://www3.nhk.or.jp/news/html/20150301/k1001...,03月03日 11時30分
2,k10010000991000,３７％の会社が「正社員が足りない」と思っている,３７％の<ruby>会社<rt>かいしゃ</rt></ruby>が「<ruby>正社員<rt...,調査会社の<org>帝国データバンク</org>がことし１月、正社員の数がちょうどいいかどう...,<ruby>調査<rt>ちょうさ</rt></ruby><ruby>会社<rt>がいしゃ</...,http://www3.nhk.or.jp/news/easy/k1001000099100...,http://www3.nhk.or.jp/news/html/20150302/k1001...,03月03日 11時30分
3,k10010001851000,ラグビーのワールドカップ １２の会場が決まる,ラグビーのワールドカップ １２の<ruby>会場<rt>かいじょう</rt></ruby>が...,２０１９年の秋、アジアで初めてのラグビーワールドカップが<plc>日本</plc>であります...,２０１９<ruby>年<rt>ねん</rt></ruby>の<ruby>秋<rt>あき</r...,http://www3.nhk.or.jp/news/easy/k1001000185100...,http://www3.nhk.or.jp/news/html/20150302/k1001...,03月03日 17時00分
4,k10010001911000,日本の顕微鏡 １００年を記念する展示会,<ruby>日本<rt>にっぽん</rt></ruby>の<ruby>顕微鏡<rt>けんびき...,１９１４年に<plc>日本</plc>で初めて顕微鏡の大量生産が始まりました。それから１００...,"１９１４<ruby>年<rt>ねん</rt></ruby>に<span class=""col...",http://www3.nhk.or.jp/news/easy/k1001000191100...,http://www3.nhk.or.jp/news/html/20150303/k1001...,03月04日 11時30分
...,...,...,...,...,...,...,...,...
6075,k10015884001000,楽天の優勝セール 安く売っているように見せた疑い,<ruby>楽天<rt>らくてん</rt></ruby>の<ruby>優勝<rt>ゆうしょう...,ことしのプロ野球は<org>東北楽天イーグルス</org>が優勝しました。このチームを持って...,ことしのプロ<ruby>野球<rt>やきゅう</rt></ruby>は<span class...,http://www3.nhk.or.jp/news/easy/k1001588400100...,http://www3.nhk.or.jp/news/html/20131107/k1001...,11月08日 16時00分
6076,k10015885731000,若田光一さんが国際宇宙ステーションに着く,<ruby>若田<rt>わかた</rt></ruby><ruby>光一<rt>こういち</r...,<per>若田</per><per>光一</per>さんたち３人の宇宙飛行士を乗せた<plc...,"<span class=""colorN""><ruby>若田<rt>わかた</rt></rub...",http://www3.nhk.or.jp/news/easy/k1001588573100...,http://www3.nhk.or.jp/news/html/20131107/k1001...,11月08日 16時00分
6077,k10015886941000,広島 原爆で亡くなった人の骨を家族に届けたい,<ruby>広島<rt>ひろしま</rt></ruby> <ruby>原爆<rt>げんばく<...,<plc>広島市</plc>の<plc>平和公園</plc>にある「原爆供養塔」には、原子爆...,"<span class=""colorL""><ruby>広島市<rt>ひろしまし</rt></...",http://www3.nhk.or.jp/news/easy/k1001588694100...,http://www3.nhk.or.jp/news/html/20130708/k1001...,07月09日 16時00分
6078,k10015892741000,災害のとき 外国から来た人の役に立つアプリができる,<ruby>災害<rt>さいがい</rt></ruby>のとき <ruby>外国<rt>がい...,地震や津波などの災害が起きたとき、旅行などで<plc>日本</plc>へ来た外国人の役に立つ...,<ruby>地震<rt>じしん</rt></ruby>や<ruby>津波<rt>つなみ</r...,http://www3.nhk.or.jp/news/easy/k1001589274100...,http://www3.nhk.or.jp/news/html/20141102/k1001...,11月04日 20時30分


In [13]:
easy.merge(df, how='inner', on='id')

,id,title_easy,title_easy_ruby,article_easy,article_easy_ruby,url_easy,url_normal,date_easy,title,article,genre,datePublished
0,k10010000621000,ウィリアム王子 地震や津波で亡くなった人に花を贈る,ウィリアム<ruby>王子<rt>おうじ</rt></ruby> <ruby>地震<rt>じ...,<plc>イギリス</plc>の<per>エリザベス</per>女王の孫、<per>ウィリア...,"<span class=""colorL"">イギリス</span>の<span class=""...",http://www3.nhk.or.jp/news/easy/k1001000062100...,http://www3.nhk.or.jp/news/html/20150301/k1001...,03月02日 17時30分,ウィリアム王子 震災の犠牲者に花束,イギリス王室のウィリアム王子は東日本大震災の被災地の宮城県石巻市と女川町を訪れ、震災の犠牲者...,[社会],2015-03-01T17:37
1,k10010000861000,「流しびな」で子どもが元気に大きくなるように願う,「<ruby>流<rt>なが</rt></ruby>しびな」で<ruby>子<rt>こ</r...,３月３日はひな祭りです。１日、<plc>埼玉県</plc><plc>さいたま市</plc>の...,３<ruby>月<rt>がつ</rt></ruby><ruby>３日<rt>みっか</rt>...,http://www3.nhk.or.jp/news/easy/k1001000086100...,http://www3.nhk.or.jp/news/html/20150301/k1001...,03月03日 11時30分,健やかな成長願う「流しびな」,桃の節句を前に、人形作りが盛んなさいたま市岩槻区で子どもの健やかな成長を願って紙のひな人形を...,[科学・文化],2015-03-01T23:32
2,k10010000991000,３７％の会社が「正社員が足りない」と思っている,３７％の<ruby>会社<rt>かいしゃ</rt></ruby>が「<ruby>正社員<rt...,調査会社の<org>帝国データバンク</org>がことし１月、正社員の数がちょうどいいかどう...,<ruby>調査<rt>ちょうさ</rt></ruby><ruby>会社<rt>がいしゃ</...,http://www3.nhk.or.jp/news/easy/k1001000099100...,http://www3.nhk.or.jp/news/html/20150302/k1001...,03月03日 11時30分,企業の約４０％ 「正社員不足感じる」,雇用情勢の改善傾向が続くなか、企業の４０％近くが正社員の不足を感じているという調査結果がまと...,[ビジネス],2015-03-02T04:16
3,k10010001851000,ラグビーのワールドカップ １２の会場が決まる,ラグビーのワールドカップ １２の<ruby>会場<rt>かいじょう</rt></ruby>が...,２０１９年の秋、アジアで初めてのラグビーワールドカップが<plc>日本</plc>であります...,２０１９<ruby>年<rt>ねん</rt></ruby>の<ruby>秋<rt>あき</r...,http://www3.nhk.or.jp/news/easy/k1001000185100...,http://www3.nhk.or.jp/news/html/20150302/k1001...,03月03日 17時00分,ラグビーＷ杯 会場の１２都市が決定,２０１９年に日本で開かれるラグビーワールドカップで試合会場となる１２の都市が発表され、東日本...,[スポーツ],2015-03-02T21:52
4,k10010001911000,日本の顕微鏡 １００年を記念する展示会,<ruby>日本<rt>にっぽん</rt></ruby>の<ruby>顕微鏡<rt>けんびき...,１９１４年に<plc>日本</plc>で初めて顕微鏡の大量生産が始まりました。それから１００...,"１９１４<ruby>年<rt>ねん</rt></ruby>に<span class=""col...",http://www3.nhk.or.jp/news/easy/k1001000191100...,http://www3.nhk.or.jp/news/html/20150303/k1001...,03月04日 11時30分,国産顕微鏡１００年を記念する展示会,日本で初めて、顕微鏡が量産化されてから１００年がすぎたのを記念する展示会が、３日から東京・上...,[科学・文化],2015-03-03T04:10
...,...,...,...,...,...,...,...,...,...,...,...,...
5716,k10015878911000,世界の若い人たちが技術の競争をする 日本はメダル１２個,<ruby>世界<rt>せかい</rt></ruby>の<ruby>若<rt>わか</rt>...,２２歳以下の若い人たちが、２年に１度世界中から集まって、仕事などで覚えた技術の競争をする大会...,２２<ruby>歳<rt>さい</rt></ruby><ruby>以下<rt>いか</rt>...,http://www3.nhk.or.jp/news/easy/k1001587891100...,http://www3.nhk.or.jp/news/html/20130708/k1001...,07月09日 11時30分,技能五輪 日本勢はメダル１２個,世界各地の若い技術者たちが仕事の技能を競い合う「技能五輪国際大会」がドイツで開かれ、日本は出...,[国際],2013-07-08T09:41
5717,k10015879641000,「女性は痛みを我慢しないで病院へ行って」,「<ruby>女性<rt>じょせい</rt></ruby>は<ruby>痛<rt>いた</r...,女性が生理のときのひどい痛みを我慢してそのままにしておくと、「子宮内膜症」などの病気になる可...,<ruby>女性<rt>じょせい</rt></ruby>が<ruby>生理<rt>せいり</...,http://www3.nhk.or.jp/news/easy/k1001587964100...,http://www3.nhk.or.jp/news/html/20131107/k1001...,11月08日 16時00分,重い生理痛 子宮内膜症のおそれも,重い生理痛を放っておくと子宮内膜症などの病気に進行するおそれがあるとして、医師などで作るグル...,[科学・文化],2013-11-07T17:35
5718,k10015884001000,楽天の優勝セール 安く売っているように見せた疑い,<ruby>楽天<rt>らくてん</rt></ruby>の<ruby>優勝<rt>ゆうしょう...,ことしのプロ野球は<org>東北楽天イーグルス</org>が優勝しました。このチームを持って...,ことしのプロ<ruby>野球<rt>やきゅう</rt></ruby>は<span class...,http://www3.nhk.or.jp/news/easy/k1001588400100...,http://www3.nhk.or.jp/news/html/20131107/k1001...,11月08日 16時00分,楽天セール ２０業者が不当表示か,インターネット通信販売大手の楽天は、自社が運営するサイトで行っていたプロ野球、東北楽天イーグ...,[ビジネス],2013-11-07T19:29
5719,k10015885731000,若田光一さんが国際宇宙ステーションに着く,<ruby>若田<rt>わかた</rt></ruby><ruby>光一<rt>こういち</r...,<per>若田</per><per>光一</per>さんたち３人の宇宙飛行士を乗せた<plc...,"<span class=""colorN""><ruby>若田<rt>わかた</rt></rub...",http://www3.nhk.or.jp/news/easy/k1001588573100...,http://www3.nhk.or.jp/news/html/20131107/k1001...,11月08日 16時00分,若田さんのソユーズがドッキング,宇宙飛行士の若田光一さんを乗せて打ち上げられたロシアの宇宙船「ソユーズ」は、日本時間の７日午...,"[科学・医療, ドッキング, ソユーズ, エンジン]",2013-11-07T19:35


In [10]:
data.article[0]

'野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策について、「毎日のように、ころころ変わっている」と述べ、民主党は、２０３０年代に原発稼働ゼロを目指して着実に取り組む考えを強調しました。\nこの中で野田総理大臣は、４日に公示される衆議院選挙について、「政権交代の前では到底、実現できなかった改革を前に進めていくのか、既得権益としがらみが染みついた古い政治へと時計の針を後ろに戻してしまうのかが問われる戦いだ」と述べました。そして野田総理大臣は、原子力政策について、「『これ以上、原発には依存しないでほしい』という国民の覚悟を受け止めて、民主党は２０３０年代に原発稼働ゼロを目指す方針を閣議決定した。自民党は、１０年間立ち止まって考えると言うが、それは『続原発』であり、無責任だ」と述べました。そのうえで野田総理大臣は、日本未来の党や日本維新の会が掲げる原子力政策について、「第三極、第四極、いろいろな政党が出るようだが、原発への対応もエネルギー政策も、毎日のように、ころころ変わっている。この国をどのようにしていこうとしているのか、さっぱり分からない。やっぱり最後は民主党と自民党との戦いだ」と述べました。'

In [51]:
newdf = pd.DataFrame(columns=list(data.columns) + ['unlemma', 'lemma', 'pos'])
for i, row in data.iterrows():
    tokens = tokenize(row['article'])
    newdf.loc[i] = list(row) + list(tokens)

In [52]:
newdf[['unlemma']].iat[0,0]

['<SEP>',
 '今週',
 '、',
 '学校',
 'の',
 '授業',
 'や',
 '行事',
 'の',
 '最中',
 'に',
 '、',
 '子ども',
 'たち',
 'が',
 '熱中',
 '症',
 'の',
 '症状',
 'を',
 '訴える',
 'ケース',
 'が',
 '相次い',
 'で',
 'い',
 'ます',
 '。',
 '<SEP>',
 '文部',
 '科学',
 '省',
 'は',
 '、',
 '学校',
 'で',
 'の',
 '熱中',
 '症',
 'の',
 'ほとんど',
 'が',
 '体育',
 'の',
 '授業',
 'など',
 'スポーツ',
 '活動',
 'の',
 '際',
 'に',
 '起き',
 'て',
 'いる',
 'こと',
 'から',
 '、',
 '全国',
 'の',
 '小中学校',
 'など',
 'に',
 '指針',
 'を',
 '示し',
 '、',
 '運動',
 'さ',
 'せる',
 '際',
 'は',
 '予防',
 '対策',
 'を',
 '徹底',
 'する',
 'よう',
 '求め',
 'て',
 'い',
 'ます',
 '。',
 '<SEP>',
 'それ',
 'に',
 'より',
 'ます',
 'と',
 '、',
 '気温',
 'が',
 '３',
 '０',
 '度',
 '以下',
 'で',
 'あっ',
 'て',
 'も',
 '湿度',
 'が',
 '高い',
 'とき',
 'は',
 '熱中',
 '症',
 'の',
 'リスク',
 'が',
 '増す',
 'として',
 '、',
 '運動',
 'する',
 '際',
 'は',
 '、',
 '室内',
 '外',
 'を',
 '問わ',
 'ず',
 '、',
 '必要',
 'に',
 '応じ',
 'て',
 '水分',
 'や',
 '塩分',
 'を',
 '補給',
 'できる',
 '環境',
 'を',
 '整える',
 'よう',
 'に',
 'する',
 'と',
 'し',
 'て',
 'い',
 'ます',
 '。',
 '<SEP>',
 'また',
 '、',
 '梅雨',
 '明け'

In [45]:
i = 5
print(len(newdf.loc[i].unlemma))
print(len(newdf.loc[i].lemma))
print(len(newdf.loc[i].pos))

335
335
335
